In [1]:
import numpy as np
import pandas as pd
from scipy.special import expit, logit

import bayesflow as bf

import keras

INFO:bayesflow:Using backend 'torch'
When using torch backend, we need to disable autograd by default to avoid excessive memory usage. Use

with torch.enable_grad():
    ...

in contexts where you need gradients (e.g. custom training loops).


## Simulator (self-contained)

In [2]:
N_TIME_POINTS = 5

class Simulator:
    def __init__(self, sigma_noise=0.1):
        """
        Simulator for the hierarchical AR(1) model:
            y[t] = alpha + beta y[t-1] + noise[t]
        starting from an initial value.

        Parameters:
            sigma_noise (float): noise standard deviation.
        """
        self.sigma_noise = sigma_noise
        self.initial_is_zero = False

    def __call__(self, params, n_time_points=N_TIME_POINTS):
        eta = np.array(params['eta'])
        alpha = np.array(params['alpha'])
        N = eta.size
        if eta.ndim > 1:
            raise ValueError("eta must be a 1D array.")

        # Generate noise for the increments: shape (N, n_time_points)
        noise = np.random.normal(
            loc=0,
            scale=self.sigma_noise,
            size=(N, n_time_points)
        )

        # Initialize trajectories with the initial condition
        traj = np.zeros((N, n_time_points))

        # Simulate the AR(1) process for each trajectory and each batch
        if not self.initial_is_zero:
            traj[:, 0] = noise[:, 0]
        for t in range(1, n_time_points):
            traj[:, t] = alpha + traj[:, t - 1] * eta + noise[:, t]

        return dict(observable=traj)


class Prior:
    def __init__(self):
        """
        Hierarchical prior for the AR(1) model.
        """
        self.alpha_mean = 0
        self.alpha_std = 1
        self.beta_mean = 0
        self.beta_std = 1 #0.1
        self.log_sigma_mean = 0 #np.log(0.1)
        self.log_sigma_std = 1 #0.5
        self.n_params_global = 3
        self.n_params_local = 1
        self.global_param_names = [r'$\alpha$', r'$\beta$', r'$\log \sigma$']

        # Build prior parameters as tensors.
        self.hyper_prior_means = np.array(
            [self.alpha_mean,
             self.beta_mean,
             self.log_sigma_mean],
        )
        self.hyper_prior_stds = np.array(
            [self.alpha_std,
             self.beta_std,
             self.log_sigma_std],
        )

        np.random.seed(0)
        self.simulator = Simulator()

        # Compute normalization constants
        test = self.sample(1000)
        self.norm_x_mean = np.mean(test['data'])
        self.norm_x_std = np.std(test['data'])
        self.norm_prior_global_mean = np.mean(test['global_params'], axis=0)
        self.norm_prior_global_std = np.std(test['global_params'], axis=0)
        self.norm_prior_local_mean = np.mean(test['local_params_raw'], axis=0)
        self.norm_prior_local_std = np.std(test['local_params_raw'], axis=0)

    def __call__(self, batch_size):
        return self.sample(batch_size=batch_size)

    @staticmethod
    def get_local_param_names(n_local_samples):
        return [r'$\eta_{' + str(i) + '}$' for i in range(n_local_samples)]

    def _sample_global(self):
        # Sample global parameters
        self.alpha = np.random.normal(loc=self.alpha_mean, scale=self.alpha_std)
        self.beta = np.random.normal(loc=self.beta_mean, scale=self.beta_std)
        self.log_sigma = np.random.normal(loc=self.log_sigma_mean, scale=self.log_sigma_std)
        return dict(alpha=self.alpha, beta=self.beta, log_sigma=self.log_sigma)

    def _sample_local(self, n_local_samples=1):
        # Sample local parameters
        eta_raw = np.random.normal(loc=0, scale=np.exp(self.log_sigma), size=n_local_samples)
        eta = self.transform_local_params(beta=self.beta, eta_raw=eta_raw)
        return dict(eta=eta, eta_raw=eta_raw)

    @staticmethod
    def transform_local_params(beta, eta_raw):
        # transform raw local parameters
        return 2*expit(beta + eta_raw)-1

    @staticmethod
    def back_transform_local_params(local_params):
        local_params_raw = logit((local_params + 1) / 2)
        local_params_raw[local_params_raw < -100] = -100
        local_params_raw[local_params_raw > 100] = 100
        return local_params_raw

    def sample(self, batch_size, n_local_samples=1, n_time_points=N_TIME_POINTS, get_grid=False):
        # Sample global and local parameters and simulate data
        global_params = np.zeros((batch_size, self.n_params_global))
        local_params_raw = np.zeros((batch_size, n_local_samples))
        local_params = np.zeros((batch_size, n_local_samples))
        data = np.zeros((batch_size, n_local_samples, n_time_points))

        for i in range(batch_size):
            global_sample = self._sample_global()
            local_sample = self._sample_local(n_local_samples=n_local_samples)
            sim_dict = {'alpha': global_sample['alpha'], 'eta': local_sample['eta']}
            sim = self.simulator(sim_dict, n_time_points=n_time_points)

            global_params[i] = [global_sample['alpha'], global_sample['beta'], global_sample['log_sigma']]
            local_params_raw[i] = local_sample['eta_raw']
            local_params[i] = local_sample['eta']
            data[i] = sim['observable']

        # Convert to tensors
        global_params = np.array(global_params)
        local_params = np.array(local_params)
        local_params_raw = np.array(local_params_raw)

        data = np.array(data)
        if get_grid:
            grid_size = int(np.sqrt(n_local_samples))
            data = data[:, :grid_size ** 2]
            data = data.reshape(batch_size, n_time_points, grid_size, grid_size)
            local_params = local_params[:, :grid_size ** 2]
            local_params_raw = local_params_raw[:, :grid_size ** 2]
            local_params = local_params.reshape(batch_size, grid_size, grid_size)
            local_params_raw = local_params_raw.reshape(batch_size, grid_size, grid_size)
        return dict(global_params=global_params, local_params=local_params,
                    local_params_raw=local_params_raw, data=data)

    def normalize_theta(self, theta, global_params):
        if global_params:
            return (theta - self.norm_prior_global_mean) / self.norm_prior_global_std
        return (theta - self.norm_prior_local_mean) / self.norm_prior_local_std

    def denormalize_theta(self, theta, global_params):
        if global_params:
            return theta * self.norm_prior_global_std + self.norm_prior_global_mean
        return theta * self.norm_prior_local_std + self.norm_prior_local_mean

    def normalize_data(self, x):
        return (x - self.norm_x_mean) / self.norm_x_std

## Experiment 2a: Small 4 x 4 Grids

In [3]:
prior = Prior()
sim = Simulator()

# Match budget of compositional
train_dict = prior.sample(
    batch_size=625,
    n_local_samples=16,
    n_time_points=5
)

train_dict['global_params'] = prior.normalize_theta(train_dict['global_params'], global_params=True)
train_dict['data'] = prior.normalize_data(train_dict['data'])

val_dict = prior.sample(
    batch_size=300,
    n_local_samples=16,
    n_time_points=5
)

val_dict['global_params'] = prior.normalize_theta(val_dict['global_params'], global_params=True)
val_dict['data'] = prior.normalize_data(val_dict['data'])

In [5]:
adapter = (
    bf.adapters.Adapter()
    .to_array()
    .convert_dtype("float64", "float32")
    .drop(["local_params", "local_params_raw"])
    .rename("global_params", "inference_variables")
    .rename("data", "summary_variables")
)

In [6]:
models = {
    "fm": (bf.networks.FlowMatching, {}),
    "coupling": (bf.networks.CouplingFlow, {
        "transform": "spline",
        "depth": 2
    }),
    "dm_cosine_F": (bf.networks.DiffusionModel, {
        "noise_schedule": "cosine", 
        "prediction_type": "F", }),
    "dm_cosine_v": (bf.networks.DiffusionModel, {
        "noise_schedule": "cosine", 
        "prediction_type": "velocity"}),   
    "dm_cosine_noise": (bf.networks.DiffusionModel, {
        "noise_schedule": "cosine", 
        "prediction_type": "noise"}),
}

In [ ]:
for model_name, model_packet in models.items():

    workflow_global = bf.BasicWorkflow(
        adapter=adapter,
        summary_network=bf.networks.DeepSet(summary_dim=5, dropout=0.1, depth=1),
        inference_network=model_packet[0](**model_packet[1]),
        checkpoint_filepath=f"bf_checkpoints/{model_name}_16"
    )

    history = workflow_global.fit_offline(
        train_dict, batch_size=32, epochs=500 if model_name != "coupling" else 100, validation_data=val_dict
    )

## Experiment 2b: Larger 16 x 16 Grids

In [7]:
# 10000 data points (same as compositional)
train_dict = prior.sample(
    batch_size=40,
    n_local_samples=16 * 16,
    n_time_points=5
)

train_dict['global_params'] = prior.normalize_theta(train_dict['global_params'], global_params=True)
train_dict['data'] = prior.normalize_data(train_dict['data'])

val_dict = prior.sample(
    batch_size=300,
    n_local_samples=16 * 16,
    n_time_points=5
)

val_dict['global_params'] = prior.normalize_theta(val_dict['global_params'], global_params=True)
val_dict['data'] = prior.normalize_data(val_dict['data'])

In [ ]:
for model_name, model_packet in models.items():


    workflow_global = bf.BasicWorkflow(
        adapter=adapter,
        summary_network=bf.networks.DeepSet(summary_dim=5, dropout=0.1, depth=1),
        inference_network=model_packet[0](**model_packet[1]),
        checkpoint_filepath=f"bf_checkpoints/{model_name}_256"
    )

    history = workflow_global.fit_offline(
        train_dict, batch_size=16, epochs=200 if model_name != "coupling" else 100, validation_data=val_dict
    )

## Experiment 2c: Larger 16 x 16 Grids with 10x Simulation Budget

In [23]:
train_dict = prior.sample(
    batch_size=400,
    n_local_samples=16 * 16,
    n_time_points=5
)

train_dict['global_params'] = prior.normalize_theta(train_dict['global_params'], global_params=True)
train_dict['data'] = prior.normalize_data(train_dict['data'])

In [ ]:
for model_name, model_packet in models.items():

    workflow_global = bf.BasicWorkflow(
        adapter=adapter,
        summary_network=bf.networks.DeepSet(summary_dim=5, dropout=0.1, depth=1),
        inference_network=model_packet[0](**model_packet[1]),
        checkpoint_filepath=f"bf_checkpoints/{model_name}_256_bigS"
    )

    history = workflow_global.fit_offline(
        train_dict, batch_size=16, epochs=200 if model_name != "coupling" else 100, validation_data=val_dict
    )

## Evaluation

In [12]:
# We only evaluate the coupling flow version (Heinrich et al., 2024, Habermann et al., 2024) and the v-prediction DM
# used for our compositional backbones
eval_models = {
    "coupling": (bf.networks.CouplingFlow, {
        "transform": "spline",
        "depth": 2
    }),
    "dm_cosine_v": (bf.networks.DiffusionModel, {
        "noise_schedule": "cosine", 
        "prediction_type": "velocity"}),   
}

n_post_samples = 100

# Eval data (small and large)
test_dict_16 = prior.sample(
    batch_size=100,
    n_local_samples=4 * 4,
    n_time_points=5
)
test_dict_16['global_params'] = prior.normalize_theta(test_dict_16['global_params'], global_params=True)
test_dict_16['data'] = prior.normalize_data(test_dict_16['data'])


test_dict_256 = prior.sample(
    batch_size=100,
    n_local_samples=16 * 16,
    n_time_points=5
)
test_dict_256['global_params'] = prior.normalize_theta(test_dict_256['global_params'], global_params=True)
test_dict_256['data'] = prior.normalize_data(test_dict_256['data'])


# Eval on small grids (4 x 4, matched budget)
small_results = {}
for model_name, model_packet in eval_models.items():

    approximator = keras.saving.load_model(f"bf_checkpoints/{model_name}_16/model.keras")

    test_global_samples = approximator.sample(conditions=test_dict_16, num_samples=n_post_samples)

    rmse = bf.diagnostics.metrics.root_mean_squared_error(test_global_samples, test_dict_16)['values']
    ece = bf.diagnostics.metrics.calibration_error(test_global_samples, test_dict_16)['values']
    pc = bf.diagnostics.metrics.posterior_contraction(test_global_samples, test_dict_16)['values']

    small_results[model_name] = {
        "RMSE_mean": rmse.mean(),
        "ECE_mean": ece.mean(),
        "PC_mean": pc.mean(),
        "RMSE_std": rmse.std(),
        "ECE_std": ece.std(),
        "PC_std": pc.std()
    }


# Eval on larger grid (16 x 16, matched budget)
larger_results = {}
for model_name, model_packet in eval_models.items():

    approximator = keras.saving.load_model(f"bf_checkpoints/{model_name}_256/model.keras")

    test_global_samples = approximator.sample(conditions=test_dict_256, num_samples=n_post_samples)

    rmse = bf.diagnostics.metrics.root_mean_squared_error(test_global_samples, test_dict_256)['values']
    ece = bf.diagnostics.metrics.calibration_error(test_global_samples, test_dict_256)['values']
    pc = bf.diagnostics.metrics.posterior_contraction(test_global_samples, test_dict_256)['values']

    larger_results[model_name] = {
        "RMSE_mean": rmse.mean(),
        "ECE_mean": ece.mean(),
        "PC_mean": pc.mean(),
        "RMSE_std": rmse.std(),
        "ECE_std": ece.std(),
        "PC_std": pc.std()
    }


# Eval on larger grid (16 x 16, 10x budget)
larger_results_10x = {}
for model_name, model_packet in eval_models.items():

    approximator = keras.saving.load_model(f"bf_checkpoints/{model_name}_256_bigS/model.keras")

    test_global_samples = approximator.sample(conditions=test_dict_256, num_samples=n_post_samples)

    rmse = bf.diagnostics.metrics.root_mean_squared_error(test_global_samples, test_dict_256)['values']
    ece = bf.diagnostics.metrics.calibration_error(test_global_samples, test_dict_256)['values']
    pc = bf.diagnostics.metrics.posterior_contraction(test_global_samples, test_dict_256)['values']

    larger_results_10x[model_name] = {
        "RMSE_mean": rmse.mean(),
        "ECE_mean": ece.mean(),
        "PC_mean": pc.mean(),
        "RMSE_std": rmse.std(),
        "ECE_std": ece.std(),
        "PC_std": pc.std()
    }

### Present results

In [13]:
all_results = {
    "Small (4x4)": small_results,
    "Large (16x16)": larger_results,
    "Large x10 (16x16)": larger_results_10x,
}

rows = []

for size, res in all_results.items():
    for method, metrics in res.items():
        row = {}
        for k, v in metrics.items():
            row[k] = float(v)  # unwrap VariableArray
        rows.append((size, method, row))

# build DataFrame
index = pd.MultiIndex.from_tuples(
    [(size, method) for size, method, _ in rows],
    names=["Problem size", "Method"],
)
data = [r for _, _, r in rows]
df = pd.DataFrame(data, index=index)

# combine mean and std into "mean (std)" columns
out = {}
for metric in ["RMSE", "PC", "ECE"]:
    mean_col = f"{metric}_mean"
    std_col = f"{metric}_std"
    out[metric] = [
        f"{m:.2f} ({s:.2f})"
        for m, s in zip(df[mean_col], df[std_col])
    ]

final_df = pd.DataFrame(out, index=df.index)
final_df

RMSE           PC          ECE
Problem size      Method                                            
Small (4x4)       coupling     0.10 (0.05)  0.87 (0.09)  0.02 (0.00)
                  dm_cosine_v  0.09 (0.04)  0.94 (0.04)  0.07 (0.03)
Large (16x16)     coupling     0.17 (0.06)  0.66 (0.20)  0.07 (0.03)
                  dm_cosine_v  0.19 (0.07)  0.53 (0.38)  0.03 (0.01)
Large x10 (16x16) coupling     0.07 (0.04)  0.92 (0.07)  0.06 (0.02)
                  dm_cosine_v  0.07 (0.03)  0.96 (0.03)  0.09 (0.08)